<a href="https://colab.research.google.com/github/Pk-Kolhapurkar/LLM_finetunning_wandb_mlflow/blob/main/trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets  mlflow pyyaml


  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.

In [7]:
pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.3 MB/s eta 0:00:00


In [9]:
import os
import mlflow
import mlflow.pytorch
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import yaml
import dagshub
# Load configuration
CONFIG_PATH = "configs/training_config.yaml"
def load_config(config_path):
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# Convert learning rate to float
config['learning_rate'] = float(config['learning_rate'])

# 1. Data Preprocessing (src/data_preprocessing.py)
def preprocess_data(tokenizer_name, dataset_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    dataset = load_dataset(dataset_name)

    def tokenize(batch):
        return tokenizer(batch['text'], padding=True, truncation=True)

    tokenized_dataset = dataset.map(tokenize, batched=True)
    return tokenized_dataset, tokenizer

# 2. Model Training (src/model_training.py)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def train_model(config, tokenized_dataset, tokenizer):
    # Initialize MLflow
    dagshub.init(repo_owner='prathamesh.khade20', repo_name='Mlflow_for_llm_evaluation', mlflow=True)
    # Set the experiment name
    mlflow.set_experiment("Sentiment_analysis")  # Name of the experiment in MLflow
    mlflow.set_tracking_uri("https://dagshub.com/prathamesh.khade20/Mlflow_for_llm_evaluation.mlflow")


    with mlflow.start_run():
        model = AutoModelForSequenceClassification.from_pretrained(
            config['model_name'], num_labels=config['num_labels']
        )

        training_args = TrainingArguments(
            output_dir=config['output_dir'],
            num_train_epochs=config['num_train_epochs'],
            per_device_train_batch_size=config['batch_size'],
            eval_strategy='epoch',  # Change from 'evaluation_strategy' to 'eval_strategy'
            learning_rate=config['learning_rate'],
            weight_decay=config['weight_decay'],
            save_total_limit=config['save_total_limit'],
            logging_dir=config['logging_dir'],
            logging_steps=config['logging_steps']
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=tokenized_dataset['train'],
            eval_dataset=tokenized_dataset['validation'],
            tokenizer=tokenizer
        )

        # Log parameters and start training
        mlflow.log_params({
            'model_name': config['model_name'],
            'batch_size': config['batch_size'],
            'learning_rate': config['learning_rate'],
            'num_train_epochs': config['num_train_epochs']
        })

        trainer.train()

        # Log metrics
        metrics = trainer.evaluate()
        mlflow.log_metrics({
            'eval_loss': metrics['eval_loss'],
            'eval_accuracy': metrics.get('eval_accuracy', None),
            'eval_f1': metrics.get('eval_f1', None),
            'eval_precision': metrics.get('eval_precision', None),
            'eval_recall': metrics.get('eval_recall', None)
        })

        # Log the model
        mlflow.pytorch.log_model(model, "model")

# Main Script
if __name__ == "__main__":
    # Preprocess data
    tokenized_dataset, tokenizer = preprocess_data(
        config['tokenizer_name'], config['dataset_name']
    )

    # Train model
    train_model(config, tokenized_dataset, tokenizer)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=9ed0a81b-1fde-46fe-8289-4f8cd21d1750&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=f1438c5686aab82f32f6f288e171726eb8b640d41b8fac8d3d465778463006df




Accessing as prathamesh.khade20

Initialized MLflow to track repo "prathamesh.khade20/Mlflow_for_llm_evaluation"

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.230600,0.204249,0.926500,0.927107,0.930258,0.926500
2,0.131300,0.164381,0.933500,0.932694,0.934694,0.933500
3,0.078700,0.160300,0.935500,0.935294,0.936030,0.935500


2025/01/09 07:00:37 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/01/09 07:00:58 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-12-04; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'google-genai'}
2025/01/09 07:00:58 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version lab

🏃 View run indecisive-trout-362 at: https://dagshub.com/prathamesh.khade20/Mlflow_for_llm_evaluation.mlflow/#/experiments/3/runs/dc2f4fd9c51b4bae9e612fe1064b945e
🧪 View experiment at: https://dagshub.com/prathamesh.khade20/Mlflow_for_llm_evaluation.mlflow/#/experiments/3
